## **Import Libraries**

In [ ]:
pip install s3fs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 20.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda

In [ ]:
import s3fs
import pandas as pd

# **Funções**



### **listar_s3**
Responsável por listar arquivos e pastas dentro do bucket. Retorno um dataframe estruturado por **Caminho, tipo e objeto**.

In [ ]:
def listar_s3(bucket_name,num_reg,tipo):
 """
    Lista arquivos e pastas em um bucket S3, incluindo o tamanho dos arquivos em MB.

    Args:
        bucket_name (str): O nome do bucket S3.
        num_reg (int): Quantidade de registros desejados no retorno.
        tipo (str): 'Arquivo', 'Pasta' ou None para ambos.

    Returns:
        pandas.DataFrame: DataFrame com a lista de objetos, tipo e tamanho em MB (para arquivos).
    """

    fs = s3fs.S3FileSystem(anon=True)
    dados = []

    for dirpath, dirnames, filenames in fs.walk(bucket_name):
        for subpasta in dirnames:
            dados.append({
                          "Caminho": dirpath,
                          "Tipo": "Pasta",
                          "Objeto":subpasta,
                          "Tamanho (MB)": None
                          })
        for arquivo in filenames:
            caminho_completo = f"{dirpath}/{arquivo}"
            try:
                info = fs.info(caminho_completo)
                tamanho_bytes = info['size']
                tamanho_mb = tamanho_bytes / (1024 * 1024)  # Converter bytes para MB
            except FileNotFoundError:
                tamanho_mb = None
            dados.append({"Caminho": dirpath, "Tipo": "Arquivo", "Objeto": arquivo, "Tamanho (MB)": tamanho_mb})

    df = pd.DataFrame(dados)
    df = df[df['Objeto'].notna() & (df['Objeto'] != '')]

    if tipo is None:
        df_final = df.head(num_reg)
    elif tipo == 'Pasta':
        df_final = df[df['Tipo'] == tipo].head(num_reg)
    elif tipo == 'Arquivo':
        df_final = df[df['Tipo'] == tipo].head(num_reg)

    return df_final

In [ ]:
df_list = listar_s3('s3://ons-aws-prod-opendata/',10000,'Arquivo')

### **download_list_s3**
Permite realizar o download dos dados no filesystem local de depuração da plataforma, assim facilitando a analise no formato desejado (Excel ou CSV).

In [ ]:
def download_list_s3(dataframe,formato):
  """
  Args:
        dataframe (pandas.dataframe): Dataframe com informações de pastas e arquivos recebidos na função *listar_s3*
        formato (str): formato 'CSV' ou 'Excel'.

  Returns:
        Arquivo no formato solicitado dentro do ./diretorio_atual
  """
  if formato == 'Excel':
    return dataframe.to_excel('./list_folders_files.xlsx', index=False)
  elif formato == 'CSV':
    return dataframe.to_csv('./list_folders_files.csv', index=False)

In [ ]:
download_list_s3(df_list,'Excel')

### **Download da Documetação**

- Lista no diretório do s3 todos arquivos cujo a extensão terminam em .pdf
- Realiza  o download desses pdfs no diretório local indicado na função

In [75]:
def baixar_pdfs_s3(bucket_name, diretorio_local):
    """
    Baixa todos os arquivos PDF de um bucket S3 para um diretório local,
    usando o comando walk para identificar os arquivos.


    Use case(code):
    bucket_name = 's3://ons-aws-prod-opendata/'
    diretorio_local = 'pdfs_baixados'  # Diretório local para salvar os PDFs

    baixar_pdfs_s3(bucket_name, diretorio_local)

    Args:
        bucket_name (str): O nome do bucket S3.
        diretorio_local (str): O caminho do diretório local para salvar os PDFs.
    """

    fs = s3fs.S3FileSystem(anon=True)

    # Cria o diretório local se ele não existir
    if not os.path.exists(diretorio_local):
        os.makedirs(diretorio_local)

    # Usa o comando walk para percorrer o bucket
    for dirpath, dirnames, filenames in fs.walk(bucket_name):
        for filename in filenames:
            if filename.endswith('.pdf'):
                caminho_completo = f"{dirpath}/{filename}"
                caminho_local = os.path.join(diretorio_local, filename)
                try:
                    fs.get(caminho_completo, caminho_local)
                    print(f"Arquivo {filename} baixado com sucesso")
                except Exception as e:
                    print(f"Erro ao baixar o arquivo {filename}: {e}")
